In [4]:
import gitlab
import os
import re
import pandas as pd
from gitlab.exceptions import GitlabAuthenticationError, GitlabGetError

In [5]:
# Подключение
GITLAB_URL = 'https://gitlab'
GITLAB_TOKEN = '*_*'  # Токен
PROJECT_PATH = 'dwh/dwh'
TARGET_PATH = 'stage/onec'  # Путь к папке с скриптами
BRANCH = 'master'

In [6]:
gl = gitlab.Gitlab(GITLAB_URL, private_token=GITLAB_TOKEN, ssl_verify=False)
gl.auth()  # Проверка аутентификации
# Получаем проект
project = gl.projects.get(PROJECT_PATH)

C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
# В разделе "Получаем список SQL-файлов" замените строку:
items = project.repository_tree(path=TARGET_PATH, recursive=True, ref=BRANCH)

# На следующий вариант с пагинацией:
try:
    # Вариант 1: Получить все элементы сразу (может быть медленно для больших репозиториев)
    items = project.repository_tree(path=TARGET_PATH, recursive=True, ref=BRANCH, all=True)
    
    # Или Вариант 2: Использовать итератор (более эффективно для больших репозиториев)
    # items = project.repository_tree(path=TARGET_PATH, recursive=True, ref=BRANCH, iterator=True)
    
    sql_files = [item for item in items if item['type'] == 'blob' and item['name'].lower().endswith(('.sql', '.ddl'))]
    print(f"Найдено {len(sql_files)} SQL-файлов")
    
except Exception as e:
    print(f"Ошибка при получении списка файлов: {str(e)}")
    sql_files = []

C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\Temp\ipykernel_48068\1476890113.py:2: UserWarning: Calling a `list()` method without specifying `get_all=True` or `iterator=True` will return a maximum of 20 items. Your query returned 20 of 54 items. See https://python-gitlab.readthedocs.io/en/v6.2.0/api-usage.html#pagination for more details. If this was done intentionally, then this warning can be supressed by adding the argument `get_all=False` to the `list()` call. (python-gitlab: C:\Users\yu.samoraj\AppData\Local\Temp\ipykernel_48068\1476890113.py:2)
  items = project.repository_tree(path=TARGET_PATH, recursive=True, ref=BRANCH)
C:\Users\yu.samoraj\AppData\Local\a

Найдено 54 SQL-файлов


In [8]:
# Функция для загрузки файла и преобразования в список строк
def get_file_lines(project, file_path):
    try:
        file_content = project.files.get(file_path=file_path, ref=BRANCH).decode()
        return [line.strip() for line in file_content.splitlines() if line.strip()]
    except Exception as e:
        print(f"Ошибка при чтении файла {file_path}: {str(e)}")
        return None

In [9]:
# Создаем словарь с содержимым файлов
files_dict = {}
    
for file in sql_files:
    lines = get_file_lines(project, file['path'])
    if lines is not None:
        files_dict[file['name']] = lines

C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\yu.samoraj\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gitlab.webbankir.team'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


In [10]:
# Конвертирует байтовые строки в обычные строки
def convert_bytes_to_string(files_dict):
    string_dict = {}
    for filename, lines in files_dict.items():
        string_dict[filename] = [line.decode('utf-8') if isinstance(line, bytes) else line 
                               for line in lines]
    return string_dict

In [11]:
string_dict = convert_bytes_to_string(files_dict)

In [ ]:
# Поиск

In [3]:
# _Reference - справочник
# _Document - документ 
# _Enum перечисоления 
# _Chrc план видов
#_Acc план счетов
# _InfoRg регистр сведений
# _AccumRg регистр накоплений
#  _AccRg регистр бугалтерии 
# _Node план обмена

In [ ]:
#_Ref,_Doc, _Enum, _Chrc, _Acc, _InfoRg, _Acc, _Node

In [26]:
# Префиксы для поиска (регистрозависимые, как в 1С)
prefixes = ['_ref', '_doc', '_enum', '_chrc', '_acc', '_inforg', '_node',
            'public._ref','public._doc', 'public._enum', 'public._chrc', 
            'public._acc', 'public._inforg', 'public._node']

exclude_substrings = ['_enumorder']
result = {}
for filename, sql_lines in string_dict.items():
    extracted_values = set()
    for line in sql_lines:
        words = line.lower().split()
        for word in words:
            # Проверяем, начинается ли слово с любого из префиксов
            if any(word.startswith(prefix) for prefix in prefixes):
                # Удаляем возможные запятые или другие знаки препинания в конце
                clean_word = word.strip(',;.')
                extracted_values.add(clean_word)
    result[filename] = extracted_values

In [28]:
for name, table in result.items():
    print(name)
    for item_table in table:
        if '_enumorder' not in item_table:
            print(item_table)

OFF_onec_i13096_2005.sql
_reference15533
_inforg13096
_reference20236
_inforg13071
_reference13157
_enum13165
_enum13166
_inforg22297
hand_corrects.sql
_document153
_accrg462
_accrged498
_chrc437
_document8212
_acc14
onec_13352.sql
_document13352
_document13352_vt13358
_document8212
_enum14586
_inforg14845
_chrc13413
_reference29
_document13437
_document8222
_inforg16698
onec_16400.sql
_document16400_vt16403
_document8212
_reference71
_inforg14845
_chrc13413
_chrc15050
_document16400
_document8222
_enum13202
_inforg16698
_inforg17812
onec_8212.sql
_document8212
_reference32
onec_8222.sql
_reference70
_reference45
_enum14368
_reference71
_inforg14845
_reference13451
_chrc15050
_reference14875
_document8222
_reference14635
_enum14757
_enum13202
_inforg16698
_inforg17812
onec_accrg13347.sql
_document8212
_enum16210
_reference32
_document8222
_document13326
_accumrg13347
onec_document13326.sql
_document13326
_reference19383
_enum13303
onec_document13437.sql
_enum16442
_document13437
_enum1